In [1]:
# !pip install pandas -t C:/Users/kazyb/Documents/AirData/Main/aws/pd

#### Import Libraries

In [2]:
import json
import requests
from datetime import datetime, timezone, timedelta
from decimal import Decimal
import pandas as pd

In [3]:
# # For AWS
# import boto3
# dynamodb = boto3.resource('dynamodb')
# table = dynamodb.Table('livedata')

#### Date and Time

In [4]:
now = datetime.now(timezone.utc) + timedelta(hours = 6)
ts = now.isoformat()

### JSON Part

#### AirKaz Parse

In [5]:
url_airkaz = 'https://airkaz.org'
rq = requests.get(url_airkaz)

psource = rq.text

psource = rq.text
sindex = psource.find('sensors_data')
psource = psource[sindex:]
findex = psource.find('<div class="container-fluid">')
psource = psource[:findex]
lstart = psource.find('[')+1
lfin = psource.find(']')
psource = psource[lstart:lfin].split("},")

alldata = []
json_code = json.loads(psource[0]+"}")

for e in list(range(len(psource))):
    try:
        json_code = json.loads(psource[e]+"}")
        alldata.append(json_code)
    except:
        pass

data = []

for e in alldata:
    if e['hour'] == 'now':
        data.append(e)

for air in data:
    air['ts'] = ts

datalen = list(range(len(data)))

for e in datalen:
    uid = data[e]['lat']+data[e]['lng']+data[e]['ts']+'|AirKaz'
    data[e]['pointId'] = uid
    data[e]['source'] = 'AirKaz.org'

cities = ['Алматы', 'Астана', 'Нур-Султан']
airkaz = []

for d in data:
    if d['city'] in cities and d['status'] == 'active' and d['pm25'] != None:
        airkaz.append(d)

#### KazGidroMet Parse

In [6]:
api_kgmt = 'e7c28f4e2e167304cee3418502d0e18c3a7772a61edc0b12afa71d2e4f2e3581'
url_kgmt = 'http://93.185.75.19:4003/simple/averages/last?key='

alldata = requests.get(url_kgmt+api_kgmt).json()

almaty = [27, 59, 97, 26, 28]
astana = [14, 12, 4, 8, 7]
kz = almaty+astana

kgmt = []

for d in alldata:
    if d['stationId'] in kz and d['code'] == 'PM2.5':
        kgmt.append(d)

for air in kgmt:
    air['ts'] = ts
    air['pointId'] = str(air['stationId'])+air['id']+air['ts']+'|KGMT'
    air['adr'] = 'TBA'
    air['source'] = 'КазГидроМет'
    if air['stationId'] in almaty:
        air['city'] = 'Алматы'
    elif air['stationId'] in astana:
        air['city'] = 'Астана'

In [7]:
# coordinates Алматы
kgmt_coord_al = [[27, 43.180062, 76.889797],
[59, 43.300790, 76.92561],
[97, 43.368683, 77.010878],
[26, 43.337128, 76.952744],
[28, 43.298860, 76.851442]]

# coordinates Астана
kgmt_coord_ast = [[14, 51.1572571,71.4128373],
[12, 51.1811223,71.4635968],
[4, 51.1149257,71.4263808],
[8, 51.185649, 71.344902],
[7, 51.123316, 71.484280]]

coordl = kgmt_coord_al+kgmt_coord_ast

In [8]:
for e in kgmt:
    if e['stationId'] == coordl[0][0]:
        e['lat'] = coordl[0][1]
        e['lng'] = coordl[0][2]
    elif e['stationId'] == coordl[1][0]:
        e['lat'] = coordl[1][1]
        e['lng'] = coordl[1][2]
    elif e['stationId'] == coordl[2][0]:
        e['lat'] = coordl[2][1]
        e['lng'] = coordl[2][2]
    elif e['stationId'] == coordl[3][0]:
        e['lat'] = coordl[3][1]
        e['lng'] = coordl[3][2]
    elif e['stationId'] == coordl[4][0]:
        e['lat'] = coordl[4][1]
        e['lng'] = coordl[4][2]
    elif e['stationId'] == coordl[5][0]:
        e['lat'] = coordl[5][1]
        e['lng'] = coordl[5][2]
    elif e['stationId'] == coordl[6][0]:
        e['lat'] = coordl[6][1]
        e['lng'] = coordl[6][2]
    elif e['stationId'] == coordl[7][0]:
        e['lat'] = coordl[7][1]
        e['lng'] = coordl[7][2]
    elif e['stationId'] == coordl[8][0]:
        e['lat'] = coordl[8][1]
        e['lng'] = coordl[8][2]
    elif e['stationId'] == coordl[9][0]:
        e['lat'] = coordl[9][1]
        e['lng'] = coordl[9][2]

#### AirVisual Parse

In [9]:
api_av = '07204a95-9cab-4030-9c92-585b48409ce2'
url_av = 'http://api.airvisual.com/v2/city?city=almaty&state=almaty-qalasy&country=kazakhstan&key='
data = requests.get(url_av+api_av).json()
pollist = data.get('data').get('history').get('pollution')

for e in pollist:
    e['city'] = 'Алматы'
    e['city_en'] = 'Almaty'
    e['stationId'] = '2'
    e['lat'] = 43.231535
    e['lng'] = 76.888067

url2_av = 'http://api.airvisual.com/v2/city?city=nur-sultan&state=nur-sultan&country=kazakhstan&key='
data2 = requests.get(url2_av+api_av).json()
pollist2 = data2.get('data').get('history').get('pollution')

for e in pollist2:
    e['city'] = 'Астана'
    e['city_en'] = 'Astana'
    e['stationId'] = '1'
    e['lat'] = 51.124852659802606
    e['lng'] = 71.6682643657323

airvisual = []
airvisual.append(pollist[0])
airvisual.append(pollist2[0])

for e in airvisual:
    e['pointId'] = e['ts']+e['city_en']+'|Airvisual'
    e['source'] = 'AirVisual.com'
    e['adr'] = 'TBA'
    e['p2'] = e['p2'].get('conc')
    e['ts_scrap'] = ts

In [10]:
# # Almaty
# 43.231535, 76.888067

# # Astana
# 51.124852659802606,  71.6682643657323

#### US Embassy Parse

In [11]:
us_key = 'ce61d4dc68ce4de97bfe3616304d5044bc60dc2e'
url_almaty = 'https://api.waqi.info/feed/almaty/?token='
url_astana = 'https://api.waqi.info/feed/astana/?token='

data_almaty = requests.get(url_almaty+us_key).json()
data_almatycl = dict()
data_almatycl['time'] = data_almaty['data']['time']['s']
data_almatycl['aqi'] = data_almaty['data']['aqi']
data_almatycl['pm25'] = data_almaty['data']['iaqi']['pm25']['v']
data_almatycl['ts'] = ts
data_almatycl['stationId'] = "2"
data_almatycl['city'] = 'Алматы'
data_almatycl['city_en'] = 'Almaty'
data_almatycl['lat'] = 43.221890
data_almatycl['lng'] = 76.850993

data_astana = requests.get(url_astana+us_key).json()
data_astanacl = dict()
data_astanacl['time'] = data_astana['data']['time']['s']
data_astanacl['aqi'] = data_astana['data']['aqi']
data_astanacl['pm25'] = data_astana['data']['iaqi']['pm25']['v']
data_astanacl['ts'] = ts
data_astanacl['stationId'] = "1"
data_astanacl['city'] = "Астана"
data_astanacl['city_en'] = "Astana"
data_astanacl['lat'] = 51.125307
data_astanacl['lng'] = 71.467247

us = []
us.append(data_almatycl)
us.append(data_astanacl)

for e in us:
    e['pointId'] = e['ts']+e['city_en']+'|US'
    e['source'] = 'US Embassy'
    e['adr'] = 'TBA'

In [12]:
# # Almaty US
# 43.221890, 76.850993

# # Astana US
# 51.125307, 71.467247

### AirData DataFrame

In [13]:
cols = ['pointId', 'Дата и время', 'Источник', 'Последнее обновление датчика', 'Город', 'ID датчика', 'Адрес датчика', 'Lat', 'Lng', 'PM 2.5']
airdata = pd.DataFrame(columns = cols)

#### AirKaz DF

In [14]:
airkazjsondf = pd.DataFrame(airkaz)
airkazdf = airkazjsondf[['pointId', 'ts', 'source', 'date', 'city', 'id', 'name', 'lat', 'lng', 'pm25']]
airkazdf.columns = cols
airdata = airdata.append(airkazdf)

#### KGMT DF

In [15]:
kgmtjsondf = pd.DataFrame(kgmt)
kgmtjsondf['value'] = kgmtjsondf['value']*1000
kgmtdf = kgmtjsondf[['pointId', 'ts', 'source', 'date', 'city', 'stationId', 'adr', 'lat', 'lng', 'value']]
kgmtdf.columns = cols
kgmtdf['Последнее обновление датчика'] = kgmtdf['Последнее обновление датчика'].str.replace('.000Z', '')
kgmtdf['Последнее обновление датчика'] = kgmtdf['Последнее обновление датчика'].str.replace('T', ' ')
airdata = airdata.append(kgmtdf)

C:\Users\kazyb\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\kazyb\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


#### AirVisual DF

In [16]:
avjsondf = pd.DataFrame(airvisual)
avdf = avjsondf[['pointId', 'ts_scrap', 'source', 'ts', 'city', 'stationId', 'adr', 'lat', 'lng', 'p2']]
avdf.columns = cols
avdf['Последнее обновление датчика'] = avdf['Последнее обновление датчика'].str.replace('.000Z', '')
avdf['Последнее обновление датчика'] = avdf['Последнее обновление датчика'].str.replace('T', ' ')
airdata = airdata.append(avdf)

C:\Users\kazyb\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\kazyb\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


#### US Embassy DF

In [17]:
usjsondf = pd.DataFrame(us)
usdf = usjsondf[['pointId', 'ts', 'source', 'time', 'city', 'stationId', 'adr', 'lat', 'lng', 'pm25']]
usdf.columns = cols
airdata = airdata.append(usdf)

In [18]:
airdata['PM 2.5'] = airdata['PM 2.5'].astype(float)
airdata['Lat'] = airdata['Lat'].astype(float)
airdata['Lng'] = airdata['Lng'].astype(float)
airdata['Город'] = airdata['Город'].replace('Астана', 'Нур-Султан')

### Upload to DynamoDB

In [19]:
db_json = airdata.to_json(orient='records')
db_json = json.loads(json.dumps(db_json), parse_float=Decimal)

In [20]:
# def lambda_handler(event, context):
#     table.put_item(Item=db_json)
#     return {'statusCode' : 200, 'body' : "Item Added" }

### Create Stream

In [21]:
# api_pbi = ''
# bi_json = bytes(airdata.to_json(orient='records'), encoding='utf-8')
# req = requests.post(api_pbi, bi_json)